In [9]:
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

print(torch.__version__)
print(torchaudio.__version__)

import librosa
import matplotlib.pyplot as plt
!pip install librosa
!pip install transformers torch torchaudio

2.8.0+cu126
2.8.0+cu126


In [10]:
from IPython.display import Audio
from matplotlib.patches import Rectangle
from torchaudio.utils import download_asset

torch.random.manual_seed(0)

In [11]:
# 1. 파일 경로를 로컬 m4a 파일로 변경
M4A_FILE_PATH = "/content/911.m4a"

# 2. torchaudio.load()를 사용하여 로드
try:
    SPEECH_WAVEFORM, SAMPLE_RATE = torchaudio.load(M4A_FILE_PATH)

    # 이후 특징 추출 코드는 그대로 사용
    # spectrogram_transform = T.Spectrogram(n_fft=512)
    # spec = spectrogram_transform(SPEECH_WAVEFORM)

except Exception as e:
    print(f"m4a 파일 로드 실패: {e}")

In [12]:
target_sample_rate = 16000
if SAMPLE_RATE != target_sample_rate:
    resampler = torchaudio.transforms.Resample(orig_freq=SAMPLE_RATE, new_freq=target_sample_rate)
    resampled_waveform = resampler(SPEECH_WAVEFORM)
else:
    resampled_waveform = SPEECH_WAVEFORM

In [ ]:
import torch
import torchaudio

from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
input_values = processor(resampled_waveform.squeeze(), return_tensors="pt", sampling_rate=target_sample_rate).input_values

In [15]:
with torch.no_grad():
    logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.batch_decode(predicted_ids)

In [16]:
print("-" * 20)
print(f"인식된 문장: {transcription[0]}")
print("-" * 20)

--------------------
인식된 문장: NINE ONE ONE WHAT IS YOUR EMERGENCY
--------------------
